# Data

Let's have a look at the data first

In [ ]:
import os
from pathlib import Path

input_data_path = Path(os.environ.get('INPUT_DATA_PATH', '.'))
output_data_path = Path(os.environ.get('OUTPUT_DATA_PATH', '.'))

train_file = str(input_data_path / "data_train.npz")
test_file = str(input_data_path / "data_test.npz")
prediction_file = str(output_data_path / "data_test_prediction.npz")


if not (os.path.isfile(train_file) and
        os.path.isfile(test_file)):
    if not os.path.isfile("input_public_data.zip"):
        !wget https://codalab.coresearch.club/my/datasets/download/37304c34-1d4a-4f43-bcb2-1fdeb37c5cba -O input_public_data.zip
    !unzip -n input_public_data.zip

In [ ]:
import numpy as np

In [ ]:
data_real = np.load(train_file, allow_pickle=True)

# This is the calorimeter response:
energy = data_real['EnergyDeposit']

# These are the quantities we want to predict
momentum = data_real['ParticleMomentum'][:,:2]
coordinate = data_real['ParticlePoint'][:,:2]

In [ ]:
print('energy.shape:', energy.shape)
print('momentum.shape:', momentum.shape)
print('coordinate.shape:', coordinate.shape)

So, we have images of 30x30 pixels and we want to predict 4 numbers for each of them: x, y, px and py.

Let's have a look at some of the images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(7, 7))
plt.subplot(221)
plt.imshow(energy[5])
plt.subplot(222)
plt.imshow(energy[50])
plt.subplot(223)
plt.imshow(energy[500])
plt.subplot(224)
plt.imshow(energy[5000]);

It's also worth knowing how the targets are distributed:

In [ ]:
plt.scatter(*momentum.T, s=5);

In [ ]:
plt.scatter(*coordinate.T, s=5);

Naive approach: can we predict the coordinates from the center of mass position of the calorimeter response?

In [ ]:
energy_density = energy / energy.sum(axis=(1, 2), keepdims=True)

cell_coords = np.stack([*np.meshgrid(
    np.arange(energy.shape[1]),
    np.arange(energy.shape[2])
)], axis=-1)[None,...]

center_of_mass = (energy_density[...,None] * cell_coords).sum(axis=(1, 2))

plt.figure(figsize=(8, 3))
plt.subplot(121)
plt.scatter(coordinate[:,0], center_of_mass[:,0], s=5)
plt.subplot(122)
plt.scatter(coordinate[:,1], center_of_mass[:,1], s=5);

Looks like the correlation isn't too strong. Maybe higher moments would give us a better picture, but we'll leave such experiments to you.

# Example solution

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import torch.optim as optim

from IPython.display import clear_output
from tqdm import tqdm

from sklearn.model_selection import train_test_split

In [ ]:
X = energy[:,None,...] # adding Channels dimension
Y = np.concatenate([coordinate, momentum], axis=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

In [ ]:
def make_torch_dataset(X, Y, batch_size, shuffle=True):
    X = torch.tensor(X).float()
    Y = torch.tensor(Y).float()
    ds = utils.TensorDataset(X, Y)
    return torch.utils.data.DataLoader(
        ds, batch_size=batch_size,
        pin_memory=True, shuffle=shuffle
    )

BATCH_SIZE = 1024

ds_train = make_torch_dataset(X_train, Y_train, BATCH_SIZE)
ds_val = make_torch_dataset(X_val, Y_val, BATCH_SIZE, shuffle=False)

In [ ]:
#### do we need this Conv block?
import torch.nn as nn
import torch.nn.functional as F


# We are going to build a model from several convolutional blocks.
# I.e. it's going to be:
#
#       [Conv2d -> Conv2d -> MaxPool2d] x 4
#     
# So why don't we define such a block as a separate Module?
class ConvBlock(nn.Module):
    def __init__(self,
                 in_channels,     # <== number of input channels to the 1st convolution
                 interm_channels, # <== outputs of the 1st / inputs of the 2nd convolution
                 out_channels,    # <== outputs of the 2nd convolution
                 use_batchnorm,   # <== whether we'll use batchnorm
                 initialization,  # <== function that'll initialize the weights
                 pad_1 = 1, pad_2 = 1):    # <== padd at 1st and 2nd conv 
        # First we run the base class constructor
        super(ConvBlock, self).__init__()

        # And then define all the layers used within a block
        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=interm_channels,
                               kernel_size=3, padding=pad_1)
        self.conv2 = nn.Conv2d(in_channels=interm_channels,
                               out_channels=out_channels,
                               kernel_size=3, padding=pad_2)
        self.pool = nn.MaxPool2d(2, 2)

        self.use_batchnorm = use_batchnorm
        if use_batchnorm:
            self.bn1 = nn.BatchNorm2d(interm_channels)
            self.bn2 = nn.BatchNorm2d(out_channels)

        # If initialization function provided, call it on the weights of the model
        if initialization is not None:
            initialization(self.conv1.weight)
            initialization(self.conv2.weight)

    def forward(self, x):
        x = self.conv1(x)
        if self.use_batchnorm:
            x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        if self.use_batchnorm:
            x = self.bn2(x)
        x = F.relu(x)

        x = self.pool(x)
        return x

In [ ]:
# Disclaimer: this might not be the best architecture for the task

# class Regressor(nn.Module):
#     def __init__(self):
#         super(Regressor, self).__init__()
#         self.conv1 = Conv2d(in_channels=1,
#                                out_channels=3,
#                                kernel_size=7)
#         self.pool = nn.MaxPool2d((4, 4))
#         self.conv2 = nn.Conv2d(in_channels=3,
#                                out_channels=8,
#                                kernel_size=4)

#         self.fc1 = nn.Linear(3 * 3 * 8, 32)
#         self.fc2 = nn.Linear(32, 2 + 2)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool(x)
#         x = F.relu(self.conv2(x))
#         x = x.view(len(x), -1)

#         x = F.relu(self.fc1(x))

#         return self.fc2(x)


class Regressor(nn.Module):
    def __init__(self, 
                initialization=(lambda w: torch.nn.init.kaiming_normal_(w, nonlinearity='relu')),
                use_batchnorm = True,
                drop = 0.04):
        super(Regressor, self).__init__()
        self.drop = drop
        if self.drop is not None:
            self.drop_lay = nn.Dropout(p=drop)
        
        # Convolutional layers:                                         # 1x30x30 (Channels x height x width)
        self.conv1 = ConvBlock(1, 4, 8, use_batchnorm, initialization, pad_1 = 2, pad_2 = 1) 
        self.conv2 = ConvBlock(8, 16, 32, use_batchnorm, initialization, pad_1 = 1, pad_2 = 1)
        self.conv3 = ConvBlock(32, 32, 64, use_batchnorm, initialization, pad_1 = 1, pad_2 = 1)
        self.conv4 = ConvBlock(64, 64, 64, use_batchnorm, initialization, pad_1 = 1, pad_2 = 1)
#         self.conv5 = ConvBlock(64, 64, 64, use_batchnorm, initialization, pad_1 = 1, pad_2 = 1) ## extra




        self.fc1 = nn.Linear(64*2*2, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 4)
   
    
#         self.fc1 = nn.Linear(64*2*2, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 16)
#         self.fc4 = nn.Linear(16, 4)



        
        # If initialization function provided, call it on the weights of the model
        if initialization is not None:
            initialization(self.fc1.weight)
            initialization(self.fc2.weight)
            initialization(self.fc3.weight)
#             initialization(self.fc4.weight)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
#         x = self.conv5(x)
        

        x = x.view(len(x), -1)
#         x = x.flatten(1)
        x = F.relu(self.fc1(x))
        if self.drop is not None:
            x = self.drop_lay(x)
        x = F.relu(self.fc2(x))
#         if self.drop is not None:
#             x = self.drop_lay(x)
#         x = F.relu(self.fc3(x))
#         if self.drop is not None:
#             x = self.drop_lay(x)
        x = self.fc3(x)
        return x





In [ ]:
device = torch.device('cuda:0') ## change this for submission
# device = torch.device('cpu')
# device
regressor = Regressor(initialization = None, drop =None).to(device)
learning_rate = 2.0e-3
opt = optim.Adam(regressor.parameters(), lr=learning_rate)
# Disclaimer: this might not be the best loss function for this task.
# loss_fn = torch.nn.L1Loss().to(device)
loss_fn = torch.nn.MSELoss().to(device)

In [ ]:
def metric_relative_mse(y_true, y_pred):
    return (
        (y_true - y_pred).pow(2).mean(dim=0) / y_true.pow(2).mean(dim=0)
    )

def metric_relative_mse_total(y_true, y_pred):
    return metric_relative_mse(y_true, y_pred).sum()

In [ ]:
def run_training(epochs=5):
    losses_train = []
    losses_val = []
    metrics_train = []
    metrics_val = []
    per_component_metrics_train = []
    per_component_metrics_val = []

    for epoch in tqdm(range(epochs)):
        for batch_X, batch_Y in ds_train:
            batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

            pred = regressor(batch_X)
            loss = loss_fn(pred, batch_Y)

            opt.zero_grad()
            loss.backward()
            opt.step()

            losses_train.append(loss.item())
            metrics_train.append(
                metric_relative_mse_total(batch_Y, pred).item()
            )

            per_component_metrics_train.append(
                metric_relative_mse(batch_Y, pred).detach().cpu().numpy()
            )

        avg_loss, avg_metrics, avg_per_component_metrics = [], [], []
        for batch_X, batch_Y in ds_val:
            batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

            pred = regressor(batch_X)
            loss = loss_fn(pred, batch_Y)

            avg_loss.append(loss.item())
            avg_metrics.append(
                metric_relative_mse_total(batch_Y, pred).item()
            )
            avg_per_component_metrics.append(
                metric_relative_mse(batch_Y, pred).detach().cpu().numpy()
            )
        losses_val.append(np.mean(avg_loss))
        metrics_val.append(np.mean(avg_metrics))
        per_component_metrics_val.append(
            np.mean(avg_per_component_metrics, axis=0)
        )


        clear_output()
        plt.figure(figsize=(18, 4.5))

        plt.subplot(131)

        plt.title("Loss")
        plt.plot(losses_train, label='train')
        plt.plot(
            np.linspace(0, len(losses_train), len(losses_val), endpoint=False),
            losses_val, label='val'
        )
        plt.legend()

        plt.subplot(132)

        plt.title("Metric (per component)")
        ms_train = np.array(per_component_metrics_train).T
        ms_val = np.array(per_component_metrics_val).T
        for i, (m_train, m_val, color) in enumerate(zip(ms_train,
                                                        ms_val,
                                                        plt.rcParams['axes.prop_cycle'])):
            plt.plot(m_train, label=f'train (component {i})', c=color['color'])
            plt.plot(
                np.linspace(0, len(m_train), len(m_val), endpoint=False),
                m_val, '--', label=f'val (component {i})', c=color['color']
            )
        plt.legend()

        plt.subplot(133)

        plt.title("Metric (total)")
        plt.plot(metrics_train, label='train')
        plt.plot(
            np.linspace(0, len(metrics_train), len(metrics_val), endpoint=False),
            metrics_val, label='val'
        )
        plt.legend()
        plt.show()
    print(f"Minimum val. metrics = {np.min(metrics_val)}")

In [ ]:
#0.3968
#0.3843
#0.4461
#0.43
#0.4487

In [ ]:
run_training(100)

In [ ]:
data_test = np.load(test_file, allow_pickle=True)
X_test = data_test['EnergyDeposit'][:,None,...]

In [ ]:
prediction_test = regressor(torch.tensor(X_test, device=device).float()).cpu()

In [ ]:
coordinate_test, momentum_test = (
    prediction_test.detach().numpy()[:, :2],
    prediction_test.detach().numpy()[:, 2:],
)

In [ ]:
np.savez_compressed(prediction_file,
                    ParticlePoint=coordinate_test,
                    ParticleMomentum=momentum_test)